In [1]:
import pandas as pd
import pyodbc
import warnings
import numpy as np
from datetime import datetime
import os
from dotenv import load_dotenv

warnings.simplefilter('ignore')

In [2]:
class EnvConfig:
    def __init__(self):
        load_dotenv()
    
    def get(self, key: str) -> str:

        value = os.getenv(key)
        if not value:
            print(f"Warning: {key} not found in environment")
        return value
    
    def is_configured(self) -> bool:
        required_keys = ["DB_SERVER", "DB_NAME", "DB_USER", "DB_PASSWORD"]
        return all(self.get(key) for key in required_keys)

In [3]:
env = EnvConfig()

FROM_DB_PATH = r"..\\..\\..\\Data\\Raw\\Sqlite\\AdventureWorks.mdb"

FROM_DB_CONN_PATH = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + FROM_DB_PATH

TO_DB_CONN_PATH = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={env.get('DB_SERVER')},1433;"
    f"DATABASE={env.get('DB_NAME')};"
    f"UID={env.get('DB_USER')};"
    f"PWD={env.get('DB_PASSWORD')}"
)

FROM_DB = pyodbc.connect(FROM_DB_CONN_PATH)
import_cursor = FROM_DB.cursor()

TO_DB = pyodbc.connect(TO_DB_CONN_PATH)
export_cursor = TO_DB.cursor()

In [4]:
tablesBACK = ["""Person_Address", "Production_BillOfMaterials","Production_ProductCategory",
          "Production_Product","Purchasing_PurchaseOrderHeader","Purchasing_PurchaseOrderDetail","Purchasing_Vendor","Sales_SalesTerritory",
          "Sales_Store","Sales_Customer","Sales_SalesOrderHeader"""]

tables = ["Purchasing_Vendor"]

In [5]:
for table in tables:
    try:
        import_cursor.execute(f"SELECT * FROM {table}")
        columns = [column[0] for column in import_cursor.description]

        rows = import_cursor.fetchall()
        if not rows:
            print(f"Geen data voor {table}")
            continue

        placeholders = ", ".join(["?" for _ in columns])
        # columns = [f"[{col}]" if col.lower() == "group" else col for col in columns]  VOOR SALES_TERRITORY
        insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({placeholders})"

        export_cursor.fast_executemany = True

        fout = 0

        for row in rows:
            try:
                export_cursor.execute(insert_query, row)
            except Exception as row_error:
                print(f"Fout bij invoegen van een rij in {table}: {row_error}")
                fout+=1

        TO_DB.commit()
        print(f"Tabel {table} voltooid. with errors: {fout}")

    except Exception as e:
        print(f"Fout bij tabel {table} + {e}")


Fout bij invoegen van een rij in Purchasing_Vendor: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__Purchasi__88984337447EBE39'. Cannot insert duplicate key in object 'dbo.Purchasing_Vendor'. The duplicate key value is (1492). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")
Fout bij invoegen van een rij in Purchasing_Vendor: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__Purchasi__88984337447EBE39'. Cannot insert duplicate key in object 'dbo.Purchasing_Vendor'. The duplicate key value is (1494). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")
Fout bij invoegen van een rij in Purchasing_Vendor: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__Purchasi__88984337447